In [4]:
%config IPCompleter.greedy=True
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import scipy
from datetime import time, timedelta
import seaborn as sns
pd.set_option('display.max_columns', 100)
from dateutil.parser import parse

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from pandas.api.types import CategoricalDtype

In [7]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return np.datetime64(t)

In [8]:
def trans_float(df):
    for i in df.columns:
        if df[i].dtype == object:
            if ((df[i]== 'None')|(df[i]== 'Null')).any():
                df[i].replace('None',np.nan,inplace = True)
                df[i].replace('Null',np.nan,inplace = True)
                df[i] = df[i].astype(float)
            else:
                df[i] = df[i].astype(float)

In [9]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return parse(t)

In [10]:
def read_largefile(path, encoding):
    test = pd.read_csv(path, sep=',',encoding = encoding,nrows= 10000)
    nan_count_list = []
    col_sel_list = []
    for i in range(len(test.columns.to_list())):
        nan_count_list.append((pd.isna(test.iloc[:,i]).sum()/10000)*100)

    for i in range(len(nan_count_list)):
        if nan_count_list[i]<=50:
            col_sel_list.append(test.columns.to_list()[i])
    
    reader = pd.read_csv(path,usecols= col_sel_list, sep=',', iterator=True,encoding = encoding)
    loop = True
    chunkSize = 100000
    chunks = []
    while loop:
        try:
            chunk = reader.get_chunk(chunkSize)
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    df = pd.concat(chunks, ignore_index=True)
    return df

In [11]:
microevents = read_largefile('../Local_hosp_data/microevents.csv', encoding = 'gb18030')

Iteration is stopped.


In [12]:
#微生物数据标注
# label_cul_name = microevents[microevents.REPORT_ITEM_NAME.str.contains('培养',na=False)]
# #label_cul_name.RESULT.value_counts().to_excel('micro_label_plagh.xlsx')
# tot_micro_label_plagh = pd.read_excel('micro_label_plagh.xlsx')
# before_micro_label_plagh = pd.read_excel('bc_results_label.xlsx')
# merge_table = pd.merge(tot_micro_label_plagh,before_micro_label_plagh, how = 'left', left_on = ['org_name'], right_on = ['Results'])
# merge_table.to_excel('micro_label_plagh_tot_tobefinished.xlsx')

In [13]:
microevents[(microevents['PATIENT_ID'] == 'S111347899')&(microevents['SUBJECT'] == '药敏-肺炎克雷伯菌')]#.SUBJECT.value_counts()

,TEST_NO,PATIENT_ID,VISIT_ID,EXECUTE_DATE,SPECIMEN,SPCM_SAMPLE_DATE_TIME,REQUESTED_DATE_TIME,RESULTS_RPT_DATE_TIME,SUBJECT,ITEM_NO,REPORT_ITEM_NAME,RESULT,RESULT_DATE_TIME
390649,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,1,阿莫西林/棒酸,I 19,2011/2/8 7:47:45
390650,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,2,阿米卡星,S 19,2011/2/8 7:47:45
390651,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,3,氨苄西林/舒巴坦,S 21,2011/2/8 7:47:45
390652,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,4,哌拉西林,R 12,2011/2/8 7:47:45
390653,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,5,头孢呋辛,R 6,2011/2/8 7:47:45
390654,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,6,头孢噻肟,R 15,2011/2/8 7:47:45
390655,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,7,亚胺培南,S 25,2011/2/8 7:47:45
390656,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,8,头孢他啶,S 26,2011/2/8 7:47:45
390657,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,9,头孢吡肟,S 22,2011/2/8 7:47:45
390658,153737,S111347899,1.0,2011/2/8 7:47:45,痰,NaN,2011/2/5 1:13:00,2011/2/8 7:47:45,药敏-肺炎克雷伯菌,10,美洛培南,S 29,2011/2/8 7:47:45


In [14]:
microevents[(microevents['PATIENT_ID'] == 'S111347899')&(microevents['RESULT'] == '肺炎克雷伯菌')]#.RESULT.value_counts()

,TEST_NO,PATIENT_ID,VISIT_ID,EXECUTE_DATE,SPECIMEN,SPCM_SAMPLE_DATE_TIME,REQUESTED_DATE_TIME,RESULTS_RPT_DATE_TIME,SUBJECT,ITEM_NO,REPORT_ITEM_NAME,RESULT,RESULT_DATE_TIME
82241,1113250740,S111347899,1.0,2011/1/30 9:08:48,痰,NaN,2011/1/25 15:27:00,2011/1/30 9:08:48,普通细菌培养+鉴定+药敏等,2,普通细菌培养+鉴定+药敏,肺炎克雷伯菌,2011/1/28 10:38:14
83196,1114054432,S111347899,1.0,2011/2/11 9:00:34,痰,NaN,2011/2/5 1:13:00,2011/2/11 9:00:34,普通细菌培养+鉴定+药敏等,2,普通细菌培养+鉴定+药敏,肺炎克雷伯菌,2011/2/8 7:47:45
83664,1114140183,S111347899,1.0,2011/2/20 9:27:12,痰,NaN,2011/2/14 11:50:00,2011/2/20 9:27:12,普通细菌培养+鉴定+药敏等,2,普通细菌培养+鉴定+药敏,肺炎克雷伯菌,2011/2/18 10:57:18


In [15]:
for i in ['EXECUTE_DATE','SPCM_SAMPLE_DATE_TIME','REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME','RESULT_DATE_TIME']:
    microevents[i] = microevents[i].apply(trans_time)

In [13]:
#microevents[microevents['PATIENT_ID'] == 'S110783085'].to_csv('test.csv',encoding = 'ANSI')

In [16]:
culture = microevents[microevents.REPORT_ITEM_NAME.str.contains('培养',na=False)]

In [17]:
ab_test = microevents[microevents['SUBJECT'].str.contains('^药敏')]

In [18]:
icustay_detail = pd.read_csv('../Local_hosp_data/icustay_detail.csv',encoding = 'gb18030')

In [19]:
for i in ['ADMISSION_DATE_TIME','DISCHARGE_DATE_TIME','TF_ADMISSION_DATE_TIME', 'TF_DISCHARGE_DATE_TIME']:
    icustay_detail[i] = icustay_detail[i].apply(trans_time)

In [20]:
ab_test_merge = pd.merge(ab_test, icustay_detail, how = 'left', on = ['PATIENT_ID'])
test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
         ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
test[(test['ICU_TYPE'] == 'ICU') &\
     (test['SUBJECT'].str.contains('肺炎'))&\
     #(test['ab_allna']==0)&\
    (test['REQUESTED_DATE_TIME'].dt.year==2014)&\
    (test['REPORT_ITEM_NAME']=='哌拉西林')].TEST_NO.drop_duplicates().sort_values(ascending = False).shape

(27,)

In [21]:
culture.rename(columns = {'RESULT':'micro_name'}, inplace= True)

In [22]:
culture.head()

,TEST_NO,PATIENT_ID,VISIT_ID,EXECUTE_DATE,SPECIMEN,SPCM_SAMPLE_DATE_TIME,REQUESTED_DATE_TIME,RESULTS_RPT_DATE_TIME,SUBJECT,ITEM_NO,REPORT_ITEM_NAME,micro_name,RESULT_DATE_TIME
0,12000021,S110236895,1.0,2009-01-15 08:45:00,痰,2009-01-11 16:46:36,2009-01-10 19:14:00,2009-01-15 08:45:00,真菌培养+鉴定+药敏,1,真菌培养+鉴定+药敏,酵母菌,2009-01-15 08:45:00
1,12000034,S111326401,1.0,2008-11-27 09:04:00,痰,NaT,2008-11-21 18:14:00,2008-11-27 09:04:00,真菌培养+鉴定+药敏,1,真菌培养+鉴定+药敏,经鉴定无真菌生长,2008-11-27 09:04:00
2,12000234,S110676645,1.0,2008-12-04 09:18:00,痰,2008-12-02 00:35:30,2008-12-01 13:50:00,2008-12-04 09:18:00,细菌培养(门诊),1,门诊细菌培养+药敏（仪器）,不动杆菌属,2008-12-04 09:18:00
3,12000234,S110676645,1.0,2008-12-04 09:18:00,痰,2008-12-02 00:35:30,2008-12-01 13:50:00,2008-12-04 09:18:00,细菌培养(门诊),2,门诊细菌培养+药敏（仪器）,洋葱伯克霍尔德菌,2008-12-04 09:18:00
4,12000469,S110183479,1.0,2008-12-11 10:46:00,引流液,2008-12-05 14:16:36,2008-12-05 13:45:00,2008-12-11 10:46:00,体液培养(需氧菌)+鉴定+药敏,1,体液培养(需氧菌)+鉴定+药敏,普通培养未生长,2008-12-11 10:46:00


In [23]:
ab_test = ab_test[['PATIENT_ID', 'SPECIMEN',
        'REQUESTED_DATE_TIME', 'RESULTS_RPT_DATE_TIME',
       'SUBJECT',  'REPORT_ITEM_NAME', 'RESULT']]

ab_test.rename(columns = {'REPORT_ITEM_NAME':'ab_name','RESULT':'ab_result'}, inplace= True)
ab_test['micro_name'] = ab_test['SUBJECT'].apply(lambda x: x.split('-')[1] if len(x.split('-')) == 2 else None)
ab_test.drop(columns = ['SUBJECT'], inplace= True)

In [24]:
ab_test['interpretation'] = ab_test['ab_result'].apply(lambda x: x.split(' ')[0] if len(x.split('  ')) == 2 else 'Not applicable')
ab_test['dilution_text'] = ab_test['ab_result'].apply(lambda x: x.split(' ')[-1])
def dilution(text):
    if re.search('性',text):
        return None
    elif re.findall('\d+\.\d+|\d+',text):
        return re.findall('\d+\.\d+|\d+',text)[-1]
    else:
        return None

ab_test['dilution_value'] = ab_test['dilution_text'].apply(lambda x: dilution(x))
ab_test['dilution_value'] = ab_test['dilution_value'].astype(float)

In [25]:
ab_test['year'] = ab_test['REQUESTED_DATE_TIME'].dt.year

In [26]:
#ab_test.micro_name.value_counts().to_csv('micro_abtest.csv',encoding = 'ANSI')

In [29]:
micro_abtest_sel = pd.read_excel('../Local_hosp_data/micro_abtest.xlsx',sheet_name='Sheet1')

In [30]:
ab_test_mer = pd.merge(ab_test, micro_abtest_sel,how = 'left', on = 'micro_name')

In [31]:
ab_test_sel = ab_test_mer[~pd.isna(ab_test_mer['org_category'])]

In [32]:
ab_test_sel.org_category.value_counts()

Enterobacteriaceae        138299
Acinetobacter spp.        107311
Pseudomonas aeruginosa     71209
Enterococcus spp.          29239
Staphylococcus aureus      17463
Name: org_category, dtype: int64

In [33]:
Pseudomonas_aeruginosa_ab = ['环丙沙星','亚胺培南','头孢吡肟', '美洛培南', '头孢他啶', '阿米卡星', '左旋氧氟沙星',
 '哌拉西林/他唑巴坦', '氨曲南', '庆大霉素', '妥布霉素']

Enterobacteriaceae_ab = ['环丙沙星', '复方新诺明', '头孢吡肟', '阿米卡星', '头孢他啶', '亚胺培南', '哌拉西林/他唑巴坦',
                         '厄它培南', '氨苄西林/舒巴坦', '庆大霉素', '氨曲南','头孢曲松', '氨苄西林', '妥布霉素', '头孢替坦', 
                         '美洛培南','阿莫西林/棒酸', '头孢呋辛', '四环素', '替加环素', '头孢呋辛酯',  '米诺环素'    
]

Acinetobacter_spp_ab = ['环丙沙星', '复方新诺明', '亚胺培南', '头孢吡肟', '头孢他啶',
 '氨苄西林/舒巴坦', '左旋氧氟沙星', '庆大霉素', '头孢曲松', '哌拉西林/他唑巴坦', '阿米卡星', '美洛培南',
 '四环素', '米诺环素']

Staphylococcus_aureus_ab = ['复方新诺明', '万古霉素', '利福平', '克林霉素', '红霉素', '庆大霉素', '环丙沙星',
 '四环素', '利奈唑烷', '喹努普汀/达福普汀', '莫西沙星','苯唑西林', '替加环素','头孢呋辛', '力奈唑烷','头孢唑啉', '磷霉素',
'青霉素',  '头孢西丁', '青霉素G', '苯唑青霉素','氨苄西林/舒巴坦', '厄它培南', '头孢吡肟', '哌拉西林', '头孢他啶', 
'亚胺培南', '美洛培南']

Enterococcus_spp_ab = ['环丙沙星','万古霉素', '氨苄西林','利奈唑烷', '喹努普汀/达福普汀',
 '左旋氧氟沙星', '莫西沙星', '替加环素', '高水平链霉素', '高水平庆大霉素', '力奈唑烷',
 '替考拉宁','亚胺培南','美洛培南']

In [34]:
def ab_sel(cat,ab_name):
    if cat == 'Enterobacteriaceae' and ab_name in Enterobacteriaceae_ab:
        return 1
    elif cat == 'Acinetobacter spp.' and ab_name in Acinetobacter_spp_ab:
        return 1
    elif cat == 'Pseudomonas aeruginosa' and ab_name in Pseudomonas_aeruginosa_ab:
        return 1    
    elif cat == 'Enterococcus spp.' and ab_name in Enterococcus_spp_ab:
        return 1
    elif cat == 'Staphylococcus aureus' and ab_name in Staphylococcus_aureus_ab:
        return 1 
    else:
        return 0

In [35]:
ab_test_sel['ab_Sel'] = ab_test_sel[['org_category','ab_name']].apply(lambda x: ab_sel(x[0],x[1]),axis= 1)

In [36]:
ab_test_sel

,PATIENT_ID,SPECIMEN,REQUESTED_DATE_TIME,RESULTS_RPT_DATE_TIME,ab_name,ab_result,micro_name,interpretation,dilution_text,dilution_value,year,org_category,ab_Sel
0,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,阿米卡星,S 18,铜绿假单胞菌,S,18,18.0,2010,Pseudomonas aeruginosa,1
1,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,哌拉西林,S 27,铜绿假单胞菌,S,27,27.0,2010,Pseudomonas aeruginosa,0
2,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,头孢他啶,S 25,铜绿假单胞菌,S,25,25.0,2010,Pseudomonas aeruginosa,1
3,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,头孢哌酮,S 25,铜绿假单胞菌,S,25,25.0,2010,Pseudomonas aeruginosa,0
4,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,头孢吡肟,S 21,铜绿假单胞菌,S,21,21.0,2010,Pseudomonas aeruginosa,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467457,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,环丙沙星,R >=4,鲍曼不动杆菌,R,>=4,4.0,2014,Acinetobacter spp.,1
467458,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,左旋氧氟沙星,R >=8,鲍曼不动杆菌,R,>=8,8.0,2014,Acinetobacter spp.,1
467459,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,四环素,R >=16,鲍曼不动杆菌,R,>=16,16.0,2014,Acinetobacter spp.,1
467460,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,呋喃妥因,>=512,鲍曼不动杆菌,,>=512,512.0,2014,Acinetobacter spp.,0


In [37]:
ab_test_sel.ab_Sel.value_counts()

1    237712
0    125809
Name: ab_Sel, dtype: int64

In [38]:
# ab_test_merge = pd.merge(ab_test_sel, icustay_detail, how = 'left', on = ['PATIENT_ID'])
# test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
#          ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
# test[(test['ICU_TYPE'] == 'ICU') &\
#      (test['micro_name'].str.contains('肺炎'))&\
#      #(test['ab_allna']==0)&\
#     (test['REQUESTED_DATE_TIME'].dt.year==2011)&\
#     (test['ab_name']=='哌拉西林/他唑巴坦')].REQUESTED_DATE_TIME.drop_duplicates().sort_values(ascending = False).shape

In [39]:
ab_test_sel_sel = ab_test_sel[ab_test_sel['ab_Sel'] == 1]

In [40]:
ab_test_sel_sel.replace({'利奈唑烷':'利奈唑胺', '力奈唑烷':'利奈唑胺'}, inplace= True)

In [41]:
for i in ab_test_sel.org_category.unique():
    print('---------------------%s---------------------'%i)
    print(ab_test_sel_sel[(ab_test_sel_sel['org_category'] == i)&(ab_test_sel_sel['interpretation'] == '')].ab_name.value_counts())

---------------------Pseudomonas aeruginosa---------------------
妥布霉素         1977
哌拉西林/他唑巴坦      71
氨曲南            15
环丙沙星            5
头孢吡肟            5
头孢他啶            5
阿米卡星            4
美洛培南            3
亚胺培南            2
左旋氧氟沙星          2
庆大霉素            1
Name: ab_name, dtype: int64
---------------------Enterococcus spp.---------------------
替加环素       1583
莫西沙星       1575
氨苄西林         32
高水平庆大霉素       7
高水平链霉素        7
环丙沙星          6
万古霉素          6
替考拉宁          1
亚胺培南          1
左旋氧氟沙星        1
Name: ab_name, dtype: int64
---------------------Enterobacteriaceae---------------------
妥布霉素         1447
头孢替坦          811
替加环素          622
氨苄西林           64
阿莫西林/棒酸        61
氨苄西林/舒巴坦       44
头孢呋辛           41
头孢呋辛酯          32
氨曲南            24
复方新诺明          24
米诺环素           18
环丙沙星           14
头孢吡肟           13
哌拉西林/他唑巴坦      11
阿米卡星            8
亚胺培南            8
头孢曲松            8
四环素             7
头孢他啶            7
厄它培南            6
美洛培南            5
庆大霉素            2
Name

In [42]:
# def impute_ris(ab_name, inter,dilution_value, org_category,year):
#     if inter == '':
#         ris_arr_year = ab_test_sel_sel[(ab_test_sel_sel['org_category'] == org_category)&\
#                      (ab_test_sel_sel['ab_name'] != ab_name)&\
#                         (ab_test_sel_sel['interpretation'] != '')&\
#                         (ab_test_sel_sel['dilution_value'] ==dilution_value)&\
#                         (ab_test_sel_sel['year'] ==year)].interpretation
#         ris_arr_all = ab_test_sel_sel[(ab_test_sel_sel['org_category'] == org_category)&\
#                         (ab_test_sel_sel['ab_name'] != ab_name)&\
#                         (ab_test_sel_sel['interpretation'] != '')&\
#                         (ab_test_sel_sel['dilution_value'] ==dilution_value)].interpretation
#         if len(ris_arr_year) > 0:
#             ris = ris_arr_year.value_counts().index[0]
#         elif len(ris_arr_all) > 0:
#             ris = ris_arr_all.value_counts().index[0]
#         else:                 
#             ris = 'Not applicable'
#     else:
#         ris = inter
                         
#     return ris

In [43]:
ab_togroup = ab_test_sel_sel[['org_category', 'ab_name',  'interpretation', 'dilution_value', 'year']]#.drop_duplicates()
ab_togroup['yes'] = 1
ab_togroup = ab_togroup[ab_togroup['interpretation'] != '']
ab_group = ab_togroup.groupby(['org_category', 'ab_name',  'dilution_value', 'year','interpretation']).count().reset_index()

ab_group_1st = ab_group.groupby(['org_category', 'ab_name',  'dilution_value', 'year']).\
apply(lambda x: x.sort_values('yes',ascending = False).head(1)).reset_index(drop =True)

ab_group_1st.drop(columns = 'yes', inplace= True)
ab_group_1st.rename(columns = {'interpretation':'interpretation_toimpute'}, inplace = True)
ab_test_sel_sel_1st = pd.merge(ab_test_sel_sel, ab_group_1st, how = 'left', on = ['org_category', 'ab_name', 'dilution_value', 'year'])
ab_test_sel_sel_1st['interpretation_imp'] = ab_test_sel_sel_1st[['interpretation','interpretation_toimpute']].\
apply(lambda x: x[1] if (x[0] == '') or (pd.isna(x[0])) else x[0],axis = 1)

In [44]:
#不考虑年份，用所有的去插补
ab_togroup_noyear = ab_test_sel_sel[['org_category', 'ab_name',  'interpretation', 'dilution_value']]#.drop_duplicates()
ab_togroup_noyear['yes'] = 1
ab_togroup_noyear = ab_togroup_noyear[ab_togroup_noyear['interpretation'] != '']
ab_group_noyear = ab_togroup_noyear.groupby(['org_category', 'ab_name',  'dilution_value', 'interpretation']).count().reset_index()


ab_group_1st_noyear = ab_group_noyear.groupby(['org_category', 'ab_name',  'dilution_value']).\
apply(lambda x: x.sort_values('yes',ascending = False).head(1)).reset_index(drop =True)

ab_group_1st_noyear.drop(columns = 'yes', inplace= True)
ab_group_1st_noyear.rename(columns = {'interpretation':'interpretation_toimpute_all'}, inplace = True)
ab_test_sel_sel_1st_all = pd.merge(ab_test_sel_sel_1st, ab_group_1st_noyear, how = 'left', on = ['org_category', 'ab_name', 'dilution_value'])

In [45]:
ab_test_sel_sel_1st_all['interpretation_imp_all'] = ab_test_sel_sel_1st_all[['interpretation_imp','interpretation_toimpute_all']].\
apply(lambda x: x[1] if (x[0] == '') or (pd.isna(x[0])) else x[0],axis = 1)

In [46]:
ab_test_sel_sel_1st_all.replace({np.nan:''}, inplace = True)

In [47]:
ab_test_sel_sel_1st_all[(ab_test_sel_sel_1st_all['org_category'] == 'Pseudomonas aeruginosa')&(ab_test_sel_sel_1st_all['ab_name'] == '妥布霉素')]#.ab_result.value_counts()

,PATIENT_ID,SPECIMEN,REQUESTED_DATE_TIME,RESULTS_RPT_DATE_TIME,ab_name,ab_result,micro_name,interpretation,dilution_text,dilution_value,year,org_category,ab_Sel,interpretation_toimpute,interpretation_imp,interpretation_toimpute_all,interpretation_imp_all
67,S115652379,伤口分泌,2016-08-05 11:25:59,2016-08-08 10:46:29,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
78,S114037874,痰,2016-08-05 18:53:54,2016-08-08 10:10:43,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
102,S115094713,痰,2016-08-05 17:22:36,2016-08-09 10:00:29,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
122,S111511215,静脉血,2016-08-06 13:51:11,2016-08-09 09:53:07,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
145,S115652379,伤口分泌,2016-08-06 10:35:54,2016-08-09 09:52:44,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237465,S115570051,尿,2016-08-03 09:43:08,2016-08-06 08:22:07,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
237506,S114798031,痰,2016-08-03 15:01:19,2016-08-07 08:18:05,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
237517,S115075113,痰,2016-08-04 10:14:15,2016-08-06 08:20:47,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S
237633,S115671050,导管,2016-08-04 09:11:13,2016-08-08 10:49:10,妥布霉素,<=1.0,铜绿假单胞菌,,<=1.0,1.0,2016,Pseudomonas aeruginosa,1,,,S,S


In [48]:
ab_test_sel_sel_1st_all = ab_test_sel_sel_1st_all[~((pd.isna(ab_test_sel_sel_1st_all['ab_result']))| (ab_test_sel_sel_1st_all['ab_result'] == '  ') | (ab_test_sel_sel_1st_all['ab_result'] == '   '))]

In [49]:
for i in ab_test_sel.org_category.unique():
    print('---------------------%s---------------------'%i)
    print(ab_test_sel_sel_1st_all[(ab_test_sel_sel_1st_all['org_category'] == i)&(ab_test_sel_sel_1st_all['interpretation_imp_all'] == '')].ab_name.value_counts())

---------------------Pseudomonas aeruginosa---------------------
妥布霉素         488
哌拉西林/他唑巴坦      2
氨曲南            2
Name: ab_name, dtype: int64
---------------------Enterococcus spp.---------------------
莫西沙星      97
替加环素      66
左旋氧氟沙星     1
Name: ab_name, dtype: int64
---------------------Enterobacteriaceae---------------------
替加环素     621
米诺环素      13
氨曲南        3
氨苄西林       2
头孢呋辛酯      2
头孢吡肟       1
Name: ab_name, dtype: int64
---------------------Staphylococcus aureus---------------------
替加环素    152
头孢唑啉      2
环丙沙星      1
哌拉西林      1
利奈唑胺      1
万古霉素      1
Name: ab_name, dtype: int64
---------------------Acinetobacter spp.---------------------
庆大霉素    1
Name: ab_name, dtype: int64


In [50]:
ab_test_merge = pd.merge(ab_test_sel, icustay_detail, how = 'left', on = ['PATIENT_ID'])
test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
         ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
test[(test['ICU_TYPE'] == 'ICU') &\
     (test['micro_name'].str.contains('肺炎'))&\
     #(test['ab_allna']==0)&\
    (test['REQUESTED_DATE_TIME'].dt.year==2011)&\
    (test['ab_name']=='哌拉西林')].REQUESTED_DATE_TIME.drop_duplicates().sort_values(ascending = False).shape

(37,)

In [51]:
def impute_ris_second(org_category, ab_name, dilution_value, inter):
    if inter == '':
        if org_category == 'Pseudomonas aeruginosa' and ab_name == '妥布霉素':
            if dilution_value <= 4:
                return 'S'
            elif dilution_value == 8:
                return 'I'
            elif dilution_value >=16:
                return 'R'
            else:
                return None
        if org_category == 'Pseudomonas aeruginosa' and ab_name == '氨曲南':
            return 'S'
        if org_category == 'Pseudomonas aeruginosa' and ab_name == '哌拉西林/他唑巴坦':
            return 'S'
        if org_category == 'Enterococcus spp.' and ab_name == '莫西沙星':
            if dilution_value <= 1:
                return 'S'
            else:
                return 'R'
        if org_category == 'Enterococcus spp.' and ab_name == '替加环素':
            if dilution_value <= 1:
                return 'S'
            else:
                return 'R'
        if org_category == 'Enterococcus spp.' and ab_name == '左旋氧氟沙星':
            return 'S' 
            
        if org_category == 'Enterobacteriaceae' and ab_name == '替加环素':
            if dilution_value >= 18:
                return 'S'
            elif dilution_value < 15:
                return 'R'                 
            else:
                return 'I'

        if org_category == 'Enterobacteriaceae' and ab_name == '米诺环素':
            if dilution_value >= 16:
                return 'S'
            elif dilution_value <= 12:
                return 'R'                 
            else:
                return 'I'

        if org_category == 'Enterobacteriaceae' and ab_name == '氨曲南':
            if dilution_value >= 21:
                return 'S'
            elif dilution_value <= 17:
                return 'R'                 
            else:
                return 'I'

        if org_category == 'Enterobacteriaceae' and ab_name == '头孢呋辛酯':
            return 'I'

        if org_category == 'Enterobacteriaceae' and ab_name == '氨苄西林':
            if dilution_value >= 17:
                return 'S'
            elif dilution_value <= 13:
                return 'R'                 
            else:
                return 'I'
            
        if org_category == 'Enterobacteriaceae' and ab_name == '头孢吡肟':
            return 'S'

        if org_category == 'Staphylococcus aureus' and ab_name == '替加环素':
            return 'Not applicable'
        
        if org_category == 'Staphylococcus aureus' and ab_name == '头孢唑啉':
            return 'I'
        if org_category == 'Staphylococcus aureus' and ab_name == '环丙沙星':
            return 'R'
        if org_category == 'Staphylococcus aureus' and ab_name == '利奈唑胺':
            return 'R'
        if org_category == 'Staphylococcus aureus' and ab_name == '万古霉素':
            return 'S'
        if org_category == 'Staphylococcus aureus' and ab_name == '哌拉西林':
            return 'S'
        if org_category == 'Acinetobacter spp.' and ab_name == '庆大霉素':
            return 'S'
    else:
        return inter

In [52]:
ab_test_sel_sel_1st_all['dilution_value'] = ab_test_sel_sel_1st_all['dilution_value'].apply(lambda x: np.nan if x== '' else float(x))

In [53]:
ab_test_sel_sel_1st_all['interpretation_final'] = ab_test_sel_sel_1st_all[['org_category', 'ab_name', 'dilution_value', 'interpretation_imp_all']].\
apply(lambda x: impute_ris_second(x[0],x[1],x[2],x[3]), axis = 1)

In [54]:
# ab_test_merge = pd.merge(ab_test_sel_sel_1st_all, icustay_detail, how = 'left', on = ['PATIENT_ID'])
# test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
#          ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
# test[(test['ICU_TYPE'] == 'ICU') &\
#      (test['micro_name'].str.contains('肺炎'))&\
#      #(test['ab_allna']==0)&\
#     (test['REQUESTED_DATE_TIME'].dt.year==2011)&\
#     (test['ab_name']=='哌拉西林/他唑巴坦')].REQUESTED_DATE_TIME.drop_duplicates().sort_values(ascending = False).shape

In [55]:
ab_test_sel_sel_1st_all['interpretation_final'] = ab_test_sel_sel_1st_all['interpretation_final'].replace({'r':'R', 's':'R'})

In [56]:
ab_test_final = ab_test_sel_sel_1st_all[ab_test_sel_sel_1st_all['interpretation_final'].isin(['R','I', 'S'])]
ab_test_final = ab_test_final[['PATIENT_ID', 'SPECIMEN', 'REQUESTED_DATE_TIME',
       'RESULTS_RPT_DATE_TIME', 'ab_name', 'ab_result', 'micro_name',
       'interpretation', 'dilution_text', 'dilution_value', 
       'org_category',  'interpretation_final']]

In [57]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return np.datetime64(t)

In [58]:
ab_test_final['REQUESTED_DATE_TIME'] = ab_test_final['REQUESTED_DATE_TIME'].apply(lambda x: trans_time(x))
ab_test_final['RESULTS_RPT_DATE_TIME'] = ab_test_final['RESULTS_RPT_DATE_TIME'].apply(lambda x: trans_time(x))

In [59]:
# ab_test_merge = pd.merge(ab_test_final, icustay_detail, how = 'left', on = ['PATIENT_ID'])
# test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
#          ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
# test[(test['ICU_TYPE'] == 'ICU') &\
#      (test['micro_name'].str.contains('肺炎'))&\
#      #(test['ab_allna']==0)&\
#     (test['REQUESTED_DATE_TIME'].dt.year==2011)&\
#     (test['ab_name']=='哌拉西林/他唑巴坦')].REQUESTED_DATE_TIME.drop_duplicates().sort_values(ascending = False).shape

In [60]:
culture_ant = pd.merge(culture, ab_test_final, how = 'left', on = ['PATIENT_ID', 'SPECIMEN', 'REQUESTED_DATE_TIME',
       'RESULTS_RPT_DATE_TIME',  'micro_name'])

In [61]:
culture_ant = pd.merge(culture_ant.drop(columns = ['org_category']),micro_abtest_sel, how = 'left', on = 'micro_name')

In [62]:
culture_ant[(~pd.isna(culture_ant['org_category']))&(pd.isna(culture_ant['interpretation_final']))].TEST_NO.drop_duplicates().count()

43444

In [63]:
culture_ant.TEST_NO.drop_duplicates().count()

185331

In [64]:
mdr_cal = ab_test_final[['PATIENT_ID','SPECIMEN','REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME','micro_name','org_category', 'ab_name','interpretation_final']].drop_duplicates()

In [65]:
MDR_CAL_cha = mdr_cal.set_index(['PATIENT_ID','SPECIMEN','REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME','micro_name','org_category', 'ab_name'], append = True).unstack(-1).reset_index()

In [66]:
MDR_CAL_cha = MDR_CAL_cha.iloc[:,1:]
MDR_CAL_cha.columns = ['PATIENT_ID','SPECIMEN','REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME','micro_name','org_category'] + MDR_CAL_cha['interpretation_final'].columns.to_list()

In [67]:
MDR_CAL_cha

,PATIENT_ID,SPECIMEN,REQUESTED_DATE_TIME,RESULTS_RPT_DATE_TIME,micro_name,org_category,万古霉素,亚胺培南,克林霉素,利奈唑胺,利福平,厄它培南,哌拉西林,哌拉西林/他唑巴坦,喹努普汀/达福普汀,四环素,复方新诺明,头孢他啶,头孢吡肟,头孢呋辛,头孢呋辛酯,头孢唑啉,头孢曲松,头孢替坦,头孢西丁,妥布霉素,左旋氧氟沙星,庆大霉素,替加环素,替考拉宁,氨曲南,氨苄西林,氨苄西林/舒巴坦,环丙沙星,磷霉素,米诺环素,红霉素,美洛培南,苯唑西林,苯唑青霉素,莫西沙星,阿米卡星,阿莫西林/棒酸,青霉素,青霉素G,高水平庆大霉素,高水平链霉素
0,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,铜绿假单胞菌,Pseudomonas aeruginosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN
1,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,铜绿假单胞菌,Pseudomonas aeruginosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,铜绿假单胞菌,Pseudomonas aeruginosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,铜绿假单胞菌,Pseudomonas aeruginosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,S110889698,痰,2010-04-14 14:53:00,2010-04-17 06:49:15,铜绿假单胞菌,Pseudomonas aeruginosa,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234992,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,鲍曼不动杆菌,Acinetobacter spp.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234993,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,鲍曼不动杆菌,Acinetobacter spp.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234994,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,鲍曼不动杆菌,Acinetobacter spp.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234995,S113598464,痰,2014-03-08 18:17:40,2014-03-11 01:43:43,鲍曼不动杆菌,Acinetobacter spp.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
ab_test_merge = pd.merge(MDR_CAL_cha, icustay_detail, how = 'left', on = ['PATIENT_ID'])
test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
         ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
test[(test['ICU_TYPE'] == 'ICU') &\
     (test['micro_name'].str.contains('肺炎'))&\
     #(test['ab_allna']==0)&\
    (test['REQUESTED_DATE_TIME'].dt.year==2011)&\
    ((test['哌拉西林/他唑巴坦']=='S')|(test['哌拉西林/他唑巴坦']=='I')|(test['哌拉西林/他唑巴坦']=='R'))][['PATIENT_ID','REQUESTED_DATE_TIME']].drop_duplicates().shape

(42, 2)

In [69]:
MDR_CAL_cha['time_hour'] = MDR_CAL_cha.REQUESTED_DATE_TIME.dt.strftime("%Y-%m-%d %H")

In [70]:
#进行转化为列之后的规整--相当于加和

#抗生素的名称列表
ab_list = ['万古霉素',
 '亚胺培南',
 '克林霉素',
 '利奈唑胺',
 '利福平',
 '厄它培南',
 '哌拉西林',
 '哌拉西林/他唑巴坦',
 '喹努普汀/达福普汀',
 '四环素',
 '复方新诺明',
 '头孢他啶',
 '头孢吡肟',
 '头孢呋辛',
 '头孢呋辛酯',
 '头孢唑啉',
 '头孢曲松',
 '头孢替坦',
 '头孢西丁',
 '妥布霉素',
 '左旋氧氟沙星',
 '庆大霉素',
 '替加环素',
 '替考拉宁',
 '氨曲南',
 '氨苄西林',
 '氨苄西林/舒巴坦',
 '环丙沙星',
 '磷霉素',
 '米诺环素',
 '红霉素',
 '美洛培南',
 '苯唑西林',
 '苯唑青霉素',
 '莫西沙星',
 '阿米卡星',
 '阿莫西林/棒酸',
 '青霉素',
 '青霉素G',
 '高水平庆大霉素',
 '高水平链霉素']

#定义转化函数
def ab_con(df):
    for i in ab_list:
        if (df[i]== 'S').any():
            df[i] = 'S'
        if (df[i]== 'I').any():
            df[i] = 'I'
        if (df[i]== 'R').any():
            df[i] = 'R'
    return df

MDR_CAL_test = MDR_CAL_cha.copy()
#grouped = MDR_CAL_test.groupby(['subject_id','micro_specimen_id','org_name'])
MDR_CAL_test_re = MDR_CAL_test.groupby(['PATIENT_ID','SPECIMEN','time_hour','micro_name']).apply(ab_con)
MDR_CAL_conc = MDR_CAL_test_re.drop_duplicates()

In [71]:
# #进行转化为列之后的规整--相当于加和

# #抗生素的名称列表
# ab_list = MDR_CAL_cha.columns.to_list()
# del ab_list[0:6]

# #定义转化函数
# def ab_con(df):
#     for i in ab_list:
#         if (df[i]== 'S').any():
#             df[i] = 'S'
#         elif (df[i]== 'I').any():
#             df[i] = 'I'
#         elif (df[i]== 'R').any():
#             df[i] = 'R'
#         else:
#             df[i] = np.nan
#     return df

# MDR_CAL_test = MDR_CAL_cha.copy()
# #grouped = MDR_CAL_test.groupby(['subject_id','micro_specimen_id','org_name'])
# MDR_CAL_test_re = MDR_CAL_test.groupby(['PATIENT_ID','SPECIMEN','REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME','micro_name','org_category']).apply(ab_con)
# MDR_CAL_conc = MDR_CAL_test_re.drop_duplicates()

In [72]:
#5个不同组别的微生物的名称
org_pieces = dict(list(micro_abtest_sel['micro_name'].groupby(micro_abtest_sel['org_category'])))

St_name = org_pieces['Staphylococcus aureus'].tolist()
Eb_name = org_pieces['Enterobacteriaceae'].tolist()
Ec_name = org_pieces['Enterococcus spp.'].tolist()
Ps_name = org_pieces['Pseudomonas aeruginosa'].tolist()
Ac_name = org_pieces['Acinetobacter spp.'].tolist()

In [73]:
Eb_exc_amino_list = ['雷氏普罗威登斯菌', '普罗威登斯菌属']
# Eb_exc_cefazolin_list = [
# ]
Eb_exc_cefuroxime_list = [
    '摩氏摩根菌','摩氏摩根菌摩根亚种','奇异变形菌', '彭氏变形菌', '普通变形菌',
    '普通变形菌群','普通变形菌群/彭氏变', '粘质沙雷菌'
]
Eb_exc_cefotetan_list = [
 '弗氏柠檬酸杆菌','产气肠杆菌','阴沟肠杆菌'
]
Eb_exc_tigecycline_list = [
 '摩氏摩根菌','摩氏摩根菌摩根亚种','彭氏变形菌', '普通变形菌',
    '普通变形菌群','普通变形菌群/彭氏变','雷氏普罗威登斯菌', '普罗威登斯菌属'
]
Eb_exc_ampicillin_list = [
    '克氏柠檬酸杆菌', '弗氏柠檬酸杆菌',
    '产气肠杆菌', '阴沟肠杆菌', '肺炎克雷伯菌','肺炎克雷伯菌肺炎亚种','肺炎克雷伯菌臭鼻亚种',
    '摩氏摩根菌','摩氏摩根菌摩根亚种', '彭氏变形菌', '普通变形菌',
    '普通变形菌群','普通变形菌群/彭氏变','雷氏普罗威登斯菌', '普罗威登斯菌属', '粘质沙雷菌'
]

Eb_exc_amoxicillin_list = [
 '弗氏柠檬酸杆菌','产气肠杆菌', '阴沟肠杆菌',
    '摩氏摩根菌','摩氏摩根菌摩根亚种', '雷氏普罗威登斯菌', '普罗威登斯菌属', '粘质沙雷菌'
]
Eb_exc_ampicillinsulbactam_list = [
'弗氏柠檬酸杆菌','克氏柠檬酸杆菌', '产气肠杆菌', '阴沟肠杆菌',
   '摩氏摩根菌摩根亚种', '雷氏普罗威登斯菌',  '粘质沙雷菌'
]
Eb_exc_teracycline_list = [
 '摩氏摩根菌','摩氏摩根菌摩根亚种','彭氏变形菌', '普通变形菌',
    '普通变形菌群','普通变形菌群/彭氏变','雷氏普罗威登斯菌', '普罗威登斯菌属'
]

In [74]:
def MDR(df):
    #金黄色葡萄球菌
    if df['micro_name'] in St_name:
        St_dict = {}
        St_sum = 0
        St_ab_list = []
        #氨基糖苷-庆大霉素
        if df['庆大霉素'] == 'I' or df['庆大霉素'] == 'R':
            St_dict['Aminoglycosides'] = 1
        #安沙霉素-利福平
        if df['利福平'] == 'I' or df['利福平'] == 'R':
            St_dict['Ansamycins'] = 1
        #抗MRSA 头孢菌素类    
        #抗葡萄球菌β内酰胺或头孢
        if (df[['苯唑西林',  '替加环素', '头孢呋辛', '头孢唑啉', '青霉素',  '头孢西丁',\
                '青霉素G','苯唑青霉素','氨苄西林/舒巴坦',  '厄它培南', '头孢吡肟',\
                '哌拉西林', '头孢他啶', '亚胺培南', '美洛培南']] == 'I').any() or \
            (df[['苯唑西林',  '替加环素', '头孢呋辛', '头孢唑啉', '青霉素',  '头孢西丁',\
                '青霉素G','苯唑青霉素','氨苄西林/舒巴坦',  '厄它培南', '头孢吡肟',\
                '哌拉西林', '头孢他啶', '亚胺培南', '美洛培南']] == 'R').any():
            St_dict['LactamsCephamycins'] = 1
        #氟喹诺酮-环丙沙星、莫西沙星
        if (df[['环丙沙星','莫西沙星']]=='I').any() or (df[['环丙沙星','莫西沙星']]=='R').any():
            St_dict['Fluoroquinolones'] = 1
        #叶酸途径抑制剂-甲氧苄啶 复方新诺明
        if df['复方新诺明']=='I' or df['复方新诺明']=='R':
            St_dict['Folate'] = 1
        #Fucidanes
        #糖肽类-万古霉素
        if df['万古霉素'] == 'I' or df['万古霉素'] == 'R':
            St_dict['Glycopeptides'] = 1
        #甘氨环素
        if df['替加环素'] == 'I' or df['替加环素'] == 'R':
            St_dict['Glycylcyclines'] = 1
        #林可酰胺类-克林霉素
        if df['克林霉素'] == 'I' or df['克林霉素'] == 'R':
            St_dict['Lincosamides'] = 1
        #脂肽类-达托霉素
#         if df['达托霉素'] == 'I' or df['达托霉素'] == 'R':
#             St_dict['Lipopeptides'] = 1
        #大环内酯类-红霉素
        if df['红霉素'] == 'I' or df['红霉素'] == 'R':
            St_dict['Macrolides'] = 1
        #恶唑烷酮类-利奈唑胺
        if df['利奈唑胺'] == 'I' or df['利奈唑胺'] == 'R':
            St_dict['Oxazolidinones'] = 1
        #苯丙醇类
        #磷酶酸类
        if df['磷霉素'] == 'I' or df['磷霉素'] == 'R':
            St_dict['Phosphonic_acids'] = 1       
        #链阳霉素类
        if df['喹努普汀/达福普汀'] == 'I' or df['喹努普汀/达福普汀'] == 'R':
            St_dict['Streptogramins'] = 1
        #四环素类-四环素
        if df['四环素'] == 'I' or df['四环素'] == 'R':
            St_dict['Tetracyclines'] = 1

        for i in St_dict:
            St_sum += St_dict[i]
            St_ab_list.append(i)
        if St_sum>=3:
            return ";".join(St_ab_list), 1
        else:
            return ";".join(St_ab_list), 0
    #肠球菌
    elif df['micro_name'] in Ec_name:
        Ec_dict = {}
        Ec_sum = 0
        Ec_ab_list = []
        #氨基糖苷-高浓度庆大霉素
        if df['高水平庆大霉素'] == 'I' or df['高水平庆大霉素'] == 'R':
            Ec_dict['Aminoglycosides'] = 1        
        #链霉素-链霉素
        if df['高水平链霉素'] == 'I' or df['高水平链霉素'] == 'R':
            Ec_dict['Streptomycin'] = 1            
        #碳青霉烯类-亚胺培南、美罗培南
        if (df[['亚胺培南','美洛培南']] == 'I').any() or \
                (df[['亚胺培南','美洛培南']] == 'R').any():
            if df['micro_name'] != '屎肠球菌': #只有不是屎肠球菌才执行这条命令。如果是屎肠球菌，则跳过，字典里不会纳入碳青霉烯类的信息。
                Ec_dict['Carbapenems'] = 1
        #氟喹诺酮-环丙沙星、左氧氟沙星、莫西沙星
        if (df[['环丙沙星', '左旋氧氟沙星', '莫西沙星']]=='I').any() or \
                (df[['环丙沙星', '左旋氧氟沙星', '莫西沙星']]=='R').any():
            Ec_dict['Fluroquinolones'] = 1
        #糖肽类-万古霉素
        if (df[['万古霉素', '替考拉宁']] == 'I').any() or (df[['万古霉素', '替考拉宁']] == 'R').any():
            Ec_dict['Glycopeptides'] = 1
        #甘氨环素类
        if df['替加环素'] == 'I' or df['替加环素'] == 'R':
            Ec_dict['Glycylcyclines'] = 1           
        #脂肽类-达托霉素
#         if df['达托霉素'] == 'I' or df['达托霉素'] == 'R':
#             Ec_dict['Lipopeptides'] = 1
        #恶唑烷酮类-利奈唑胺
        if df['利奈唑胺'] == 'I' or df['利奈唑胺'] == 'R':
            Ec_dict['Oxazolidinones'] = 1
        #青霉素类-氨苄西林
        if df['氨苄西林'] == 'I' or df['氨苄西林'] == 'R':
            Ec_dict['Penicillins'] = 1
        #链阳霉素类
        if df['喹努普汀/达福普汀'] == 'I' or df['喹努普汀/达福普汀'] == 'R':
            if df['micro_name'] != '粪肠球菌':
                Ec_dict['Streptogramins'] = 1
        #四环素类
#         if df['TETRACYCLINE'] == 'I' or df['TETRACYCLINE'] == 'R':
#                 Ec_dict['Tetracyclines'] = 1

        for i in Ec_dict:
            Ec_sum += Ec_dict[i]
            Ec_ab_list.append(i)
        if Ec_sum>=3:
            return ";".join(Ec_ab_list), 1
        else:
            return ";".join(Ec_ab_list), 0
    #肠杆菌
    elif df['micro_name'] in Eb_name:
        Eb_dict = {}
        Eb_sum = 0
        Eb_ab_list = []
        #氨基糖苷-庆大霉素、妥布霉素、阿米卡星
        if (df[['庆大霉素', '妥布霉素']] == 'I').any() \
            or (df[['庆大霉素', '妥布霉素']] == 'R').any():
            if df['micro_name'] not in Eb_exc_amino_list:
                Eb_dict['Aminoglycosides'] = 1
        if df['阿米卡星'] == 'I' or df['阿米卡星'] == 'R':
            Eb_dict['Aminoglycosides'] = 1
        #抗MRSA头孢-头孢洛林
        #抗假单胞菌青霉素+β-内酰胺酶抑制剂 - 哌拉西林-他唑巴坦
        if df['哌拉西林/他唑巴坦'] == 'I' or df['哌拉西林/他唑巴坦'] == 'R':
            Eb_dict['AntiPenicillins&Lactamaseinhibitors'] = 1
        #碳青霉烯类 - 亚胺培南、美罗培南
        if (df[['亚胺培南','美洛培南','厄它培南']] == 'I').any() or \
            (df[['亚胺培南','美洛培南','厄它培南']] == 'R').any():
            Eb_dict['Carbapenems'] = 1
        #非超广谱头孢菌素-1、2代头孢 - 头孢唑林、头孢呋辛
        if (df[['头孢呋辛','头孢呋辛酯']]=='I').any() or (df[['头孢呋辛','头孢呋辛酯']]=='R').any():
            if df['micro_name'] not in Eb_exc_cefuroxime_list:
                Eb_dict['NESCephalosporins'] = 1
        #超广谱头孢菌素-3、4代头孢 - 头孢曲松、头孢他啶、头孢吡肟
        if (df[['头孢吡肟',  '头孢他啶', '头孢曲松']]=='I').any() \
            or (df[['头孢吡肟',  '头孢他啶', '头孢曲松']]=='R').any():
            Eb_dict['ESCephalosporins'] = 1
        #头孢菌素类 Cephamycins
        if df['头孢替坦'] == 'I' or df['头孢替坦'] == 'R':
            if df['micro_name'] not in Eb_exc_cefotetan_list:
                Eb_dict['Cephamycins'] = 1
        #氟喹诺酮类 - 环丙沙星
        if df['环丙沙星'] == 'I' or df['环丙沙星'] == 'R':
            Eb_dict['Fluoroquinolones'] = 1
        #叶酸途径抑制剂-甲氧苄啶 复方新诺明
        if df['复方新诺明']=='I' or df['复方新诺明']=='R':
            Eb_dict['Folate'] = 1
        #甘氨环素类 - 替加环素
        if df['替加环素'] == 'I' or df['替加环素'] == 'R':
            if df['micro_name'] not in Eb_exc_tigecycline_list:
                Eb_dict['Glycylcyclines'] = 1        
        #单环β-内酰胺类 - 氨曲南
        if df['氨曲南']=='I' or df['氨曲南']=='R':
            Eb_dict['Monobactams'] = 1        
        #青霉素类 - 氨苄西林
        if df['氨苄西林']=='I' or df['氨苄西林']=='R':
            if df['micro_name'] not in Eb_exc_ampicillin_list:
                Eb_dict['Penicillins'] = 1 
        #青霉素类+β内酰胺酶抑制剂 - 氨苄西林-舒巴坦
        if df['阿莫西林/棒酸']=='I' or df['阿莫西林/棒酸']=='R':
            if df['micro_name'] not in Eb_exc_amoxicillin_list:
                Eb_dict['Penicillins&Lactamaseinhibitors'] = 1
        if df['氨苄西林/舒巴坦']=='I' or df['氨苄西林/舒巴坦']=='R':
            if df['micro_name'] not in Eb_exc_ampicillinsulbactam_list:
                Eb_dict['Penicillins&Lactamaseinhibitors'] = 1
        #苯丙醇类
        #磷酶酸类
        #多粘菌素类
        #四环素类-四环素
        if (df[['四环素','米诺环素']] == 'I').any() or (df[['四环素','米诺环素']] == 'R').any():
            if df['micro_name'] not in Eb_exc_teracycline_list:
                Eb_dict['Tetracyclines'] = 1
        
        for i in Eb_dict:
            Eb_sum += Eb_dict[i]
            Eb_ab_list.append(i)
        if Eb_sum>=3:
            return ";".join(Eb_ab_list), 1
        else:
            return ";".join(Eb_ab_list), 0
    #铜绿假单胞菌
    elif df['micro_name'] in Ps_name:
        Ps_dict = {}
        Ps_sum = 0
        Ps_ab_list = []
        #氨基糖苷类 - 庆大霉素、妥布霉素、阿米卡星
        if (df[['庆大霉素', '妥布霉素','阿米卡星']] == 'I').any() \
            or (df[['庆大霉素', '妥布霉素','阿米卡星']] == 'R').any():
            Ps_dict['Aminoglycosides'] = 1    
        #抗假单胞菌碳青霉烯类-亚胺培南、美罗培南
        if (df[['亚胺培南', '美洛培南']] == 'I').any() or \
                (df[['亚胺培南', '美洛培南']] == 'R').any():
            Ps_dict['Carbapenems'] = 1
        #抗假单胞菌头孢菌素类-头孢他啶、头孢吡肟
        if (df[['头孢他啶','头孢吡肟']] == 'I').any() or \
                (df[['头孢他啶','头孢吡肟']] == 'R').any():
            Ps_dict['Cephalosporins'] = 1      
        #抗假单胞菌氟喹诺酮类-环丙沙星、左氧氟沙星
        if (df[['环丙沙星','左旋氧氟沙星']]=='I').any() or \
                (df[['环丙沙星','左旋氧氟沙星']]=='R').any():
            Ps_dict['Fluroquinolones'] = 1
        #抗假单胞菌青霉素类+β-内酰胺酶抑制剂
        if df['哌拉西林/他唑巴坦'] == 'I' or df['哌拉西林/他唑巴坦'] == 'R':
            Ps_dict['Penicillins&lactamaseinhibitors'] = 1
        #单环β-内酰胺类 - 氨曲南
        if df['氨曲南']=='I' or df['氨曲南']=='R':
            Ps_dict['Monobactams'] = 1      
        #磷霉酸类
        #多粘菌素类
        for i in Ps_dict:
            Ps_sum += Ps_dict[i]
            Ps_ab_list.append(i)
        if Ps_sum>=3:
            return ";".join(Ps_ab_list), 1
        else:
            return ";".join(Ps_ab_list), 0
    #不动杆菌属
    elif df['micro_name'] in Ac_name:
        Ac_dict = {}
        Ac_sum = 0
        Ac_ab_list = []
        #氨基糖苷类 - 庆大霉素、妥布霉素、阿米卡星
        if (df[['庆大霉素', '妥布霉素','阿米卡星']] == 'I').any() \
            or (df[['庆大霉素', '妥布霉素','阿米卡星']] == 'R').any():
            Ac_dict['Aminoglycosides'] = 1    
        #抗假单胞菌碳青霉烯类 - 亚胺培南、美罗培南
        if (df[['亚胺培南', '美洛培南']] == 'I').any() or \
                (df[['亚胺培南', '美洛培南']] == 'R').any():
            Ac_dict['Carbapenems'] = 1
        #抗假单胞菌氟喹诺酮类 - 环丙沙星、左氧氟沙星
        if (df[['环丙沙星','左旋氧氟沙星']]=='I').any() or \
                (df[['环丙沙星','左旋氧氟沙星']]=='R').any():
            Ac_dict['Fluroquinolones'] = 1
        #抗假单胞菌青霉素类+β-内酰胺酶抑制剂
        if df['哌拉西林/他唑巴坦'] == 'I' or df['哌拉西林/他唑巴坦'] == 'R':
            Ac_dict['Penicillins&lactamaseinhibitors'] = 1
        #超广谱头孢菌素类-头孢他啶、头孢吡肟
        if (df[['头孢曲松', '头孢吡肟', '头孢他啶']] == 'I').any() or \
                (df[['头孢曲松', '头孢吡肟', '头孢他啶']] == 'R').any():
            Ac_dict['ESCephalosporins'] = 1
        #叶酸途径抑制剂-甲氧苄啶 复方新诺明
        if df['复方新诺明']=='I' or df['复方新诺明']=='R':
            Ac_dict['Folate'] = 1
        #青霉素类+β内酰胺酶抑制剂 - 氨苄西林-舒巴坦
        if df['氨苄西林/舒巴坦']=='I' or df['氨苄西林/舒巴坦']=='R':
            Ac_dict['Penicillins&lactamaseinhibitors'] = 1        
        #多粘菌素类
        #四环素类
        if (df[['四环素', '米诺环素']] == 'I').any() or (df[['四环素', '米诺环素']] == 'R').any():
            Ac_dict['Tetracyclines'] = 1        
        for i in Ac_dict:
            Ac_sum += Ac_dict[i]
            Ac_ab_list.append(i)
        if Ac_sum>=3:
            return ";".join(Ac_ab_list), 1
        else:
            return ";".join(Ac_ab_list), 0
    else:
        return np.nan, np.nan

In [75]:
MDR_CAL_conc[['resistant_ab_cat','MDR']] = MDR_CAL_conc.apply(MDR,axis = 1,result_type="expand")

In [76]:
MDR_CAL_conc.MDR.value_counts()

1    15080
0     6027
Name: MDR, dtype: int64

In [77]:
culture['merge_time'] =  culture['REQUESTED_DATE_TIME'].dt.strftime("%Y-%m-%d %H").astype(str)
MDR_CAL_conc['merge_time'] = MDR_CAL_conc['REQUESTED_DATE_TIME'].dt.strftime("%Y-%m-%d %H").astype(str)

In [78]:
culture_mdr = pd.merge(culture[['TEST_NO', 'PATIENT_ID', 'VISIT_ID',  'SPECIMEN','REQUESTED_DATE_TIME', 'RESULTS_RPT_DATE_TIME',
       'SUBJECT', 'ITEM_NO', 'REPORT_ITEM_NAME', 'micro_name','merge_time']], \
        MDR_CAL_conc[['PATIENT_ID', 'SPECIMEN', 'micro_name', 'org_category', '万古霉素', '亚胺培南',
       '克林霉素', '利奈唑胺', '利福平', '厄它培南', '哌拉西林', '哌拉西林/他唑巴坦', '喹努普汀/达福普汀', '四环素',
       '复方新诺明', '头孢他啶', '头孢吡肟', '头孢呋辛', '头孢呋辛酯', '头孢唑啉', '头孢曲松', '头孢替坦',
       '头孢西丁', '妥布霉素', '左旋氧氟沙星', '庆大霉素', '替加环素', '替考拉宁', '氨曲南', '氨苄西林',
       '氨苄西林/舒巴坦', '环丙沙星', '磷霉素', '米诺环素', '红霉素', '美洛培南', '苯唑西林', '苯唑青霉素',
       '莫西沙星', '阿米卡星', '阿莫西林/棒酸', '青霉素', '青霉素G', '高水平庆大霉素', '高水平链霉素',
       'time_hour', 'resistant_ab_cat', 'MDR', 'merge_time']],\
        how = 'left', on = ['PATIENT_ID', 'SPECIMEN','micro_name', 'merge_time'])

In [79]:
#culture_mdr['REQUESTED_DATE_TIME'] = culture_mdr['REQUESTED_DATE_TIME'].apply(trans_time)

In [80]:
ab_test_merge = pd.merge(culture_mdr, icustay_detail, how = 'left', on = ['PATIENT_ID'])
test = ab_test_merge[((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
         ((ab_test_merge['REQUESTED_DATE_TIME'] - ab_test_merge['TF_DISCHARGE_DATE_TIME']).dt.total_seconds()/(3600*24)<=0)]
test[(test['ICU_TYPE'] == 'ICU') &\
     (test['micro_name'].str.contains('肺炎'))&\
     #(test['ab_allna']==0)&\
    (test['REQUESTED_DATE_TIME'].dt.year==2011)&\
    ((test['哌拉西林/他唑巴坦']=='S')|(test['哌拉西林/他唑巴坦']=='I')|(test['哌拉西林/他唑巴坦']=='R'))][['PATIENT_ID','REQUESTED_DATE_TIME']].drop_duplicates().shape

(46, 2)

In [81]:
culture_mdr = pd.merge(culture_mdr.drop(columns = ['org_category']),micro_abtest_sel, how = 'left', on = 'micro_name')

In [82]:
def pos_neg(df):
    if (df['org_category'] == 'Staphylococcus aureus') or (df['org_category'] == 'Enterococcus spp.'):
        return 1,0
    elif (df['org_category'] == 'Enterobacteriaceae') or (df['org_category'] == 'Pseudomonas aeruginosa') or  (df['org_category'] == 'Acinetobacter spp.'):
        return 0,1
    else:
        return 0,0

In [83]:
culture_mdr[['org_pos','org_neg']] = culture_mdr.apply(pos_neg, axis = 1, result_type="expand")

In [84]:
micro_sim = culture_mdr[['PATIENT_ID', 'VISIT_ID', 'SPECIMEN','REQUESTED_DATE_TIME', 'RESULTS_RPT_DATE_TIME', 'micro_name','resistant_ab_cat','MDR']].drop_duplicates()

micro_sim['MDR'].fillna(0,inplace = True)

micro_sim_firsthospmdr = micro_sim[micro_sim['MDR']==1].groupby(['PATIENT_ID','VISIT_ID']).\
apply(lambda x: x.sort_values(['RESULTS_RPT_DATE_TIME','REQUESTED_DATE_TIME']).head(1)).reset_index(drop = True)


In [85]:
micro_sim_firsthospmdr = micro_sim_firsthospmdr[['PATIENT_ID', 'VISIT_ID',\
                                                          'REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME','resistant_ab_cat','micro_name']].\
rename(columns = {'REQUESTED_DATE_TIME':'firsthospmdrcharttime',\
                  'RESULTS_RPT_DATE_TIME':'firsthospmdrstoretime',\
                  'micro_name':'firsthospmdrorgname',\
                 'resistant_ab_cat':'firsthospmdrRAB'})

In [86]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return parse(t)

In [87]:
icustay_detail = icustay_detail[icustay_detail['LOS_ICU']>0]

icustay_detail = icustay_detail[['PATIENT_ID', 'VISIT_ID', 'GENDER', 'AGE', 'ADMISSION_DATE_TIME',
       'DISCHARGE_DATE_TIME',  'ADMISSION_TYPE','ICU_TYPE', 
       'TF_ADMISSION_DATE_TIME', 'TF_DISCHARGE_DATE_TIME', 'LOS_ICU']]

In [88]:
icustay_1stmdr = pd.merge(icustay_detail, micro_sim_firsthospmdr, how = 'left', on = ['PATIENT_ID', 'VISIT_ID'])

In [89]:
culture_mdr.drop(columns = ['TEST_NO','VISIT_ID'],inplace= True)

In [90]:
cohort_micro = pd.merge(culture_mdr,icustay_1stmdr, how = 'left', on = ['PATIENT_ID'])

In [91]:
cohort_micro['microtest_days'] = round((cohort_micro['REQUESTED_DATE_TIME'] - cohort_micro['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600/24,2)
cohort_micro['microresult_days'] = round((cohort_micro['RESULTS_RPT_DATE_TIME'] - cohort_micro['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600/24,2)

In [92]:
#在住ICU期间做的微生物检测，如果有检测，则microtest_inicu为1
cohort_micro['LOS_ICU'] = cohort_micro['LOS_ICU'].astype(float)
cohort_micro['microtest_inicu'] = cohort_micro[['microtest_days','LOS_ICU']].\
apply(lambda x: 1 if (x[0]>=0) & (x[0]<=x[1]) else 0, axis = 1)

#在入icu后，出ICU前做过微生物检测的人数
cohort_micro[['PATIENT_ID','TF_ADMISSION_DATE_TIME','microtest_inicu']].drop_duplicates().microtest_inicu.value_counts()

1    27370
0    23140
Name: microtest_inicu, dtype: int64

In [93]:
#在入icu前做微生物检测，在入ICU后结果才出来的人数
cohort_micro['microtest_beforeicu'] = cohort_micro[['microtest_days','microresult_days','LOS_ICU']].\
apply(lambda x: 1 if ((x[0]<0) & (x[1]<=x[2]) & (x[1]>0)) else 0, axis = 1)

#在入icu前做微生物检测，在入ICU后，出ICU前结果出来的人数
cohort_micro[['PATIENT_ID','TF_ADMISSION_DATE_TIME','microtest_beforeicu']].drop_duplicates().microtest_beforeicu.value_counts()

0    39465
1     4213
Name: microtest_beforeicu, dtype: int64

In [94]:
#在入icu前做微生物检测，在入ICU前结果出来的人数
cohort_micro['microresult_beforeicu'] = cohort_micro[['microtest_days','microresult_days','LOS_ICU']].\
apply(lambda x: 1 if ((x[0]<0) & (x[1]<0)) else 0, axis = 1)

#在入icu前做微生物检测，在入ICU前出结果人数
cohort_micro[['PATIENT_ID','TF_ADMISSION_DATE_TIME','microresult_beforeicu']].drop_duplicates().microresult_beforeicu.value_counts()

0    36582
1     9251
Name: microresult_beforeicu, dtype: int64

In [95]:
cohort_micro[cohort_micro['microresult_beforeicu']==1].drop_duplicates().MDR.value_counts()

1.0    7592
0.0    2642
Name: MDR, dtype: int64

In [96]:
cohort_micro[cohort_micro['SUBJECT'] == '真菌培养+鉴定+药敏'].REPORT_ITEM_NAME.value_counts()

真菌培养+鉴定+药敏         15418
真菌培养+鉴定              152
真菌培养                  44
普通细菌培养+鉴定+药敏等2项       12
普通细菌培养+鉴定+药敏           4
体液仪器真菌培养               3
真菌培养+鉴定+药敏等            3
痰细菌培养+鉴定+药敏            2
Name: REPORT_ITEM_NAME, dtype: int64

In [97]:
cohort_WITHMDR = cohort_micro.copy()

In [98]:
cohort_WITHMDR['MDR'] = cohort_WITHMDR['MDR'].fillna(0)

In [100]:
cohort_WITHMDR[cohort_WITHMDR['microtest_inicu']==1].drop_duplicates().MDR.value_counts()

0.0    258948
1.0     11416
Name: MDR, dtype: int64

In [101]:
cohort_WITHMDR[['PATIENT_ID','REQUESTED_DATE_TIME','micro_name','org_category']].drop_duplicates().org_category.value_counts()

Enterobacteriaceae        20509
Acinetobacter spp.        19667
Pseudomonas aeruginosa    10282
Enterococcus spp.          6911
Staphylococcus aureus      2473
Name: org_category, dtype: int64

In [102]:
org_cat_list = ['Staphylococcus aureus', 'Enterococcus spp.', 'Enterobacteriaceae','Pseudomonas aeruginosa','Acinetobacter spp.']

In [103]:
cohort_WITHMDR['org_sel'] = cohort_WITHMDR['org_category'].apply(lambda x: 1 if x in org_cat_list else 0)

In [104]:
ab_list = ['万古霉素', '亚胺培南', '克林霉素', '利奈唑胺', '利福平', '厄它培南', '哌拉西林',
       '哌拉西林/他唑巴坦', '喹努普汀/达福普汀', '四环素', '复方新诺明', '头孢他啶', '头孢吡肟', '头孢呋辛',
       '头孢呋辛酯', '头孢唑啉', '头孢曲松', '头孢替坦', '头孢西丁', '妥布霉素', '左旋氧氟沙星', '庆大霉素',
       '替加环素', '替考拉宁', '氨曲南', '氨苄西林', '氨苄西林/舒巴坦', '环丙沙星', '磷霉素', '米诺环素', '红霉素',
       '美洛培南', '苯唑西林', '苯唑青霉素', '莫西沙星', '阿米卡星', '阿莫西林/棒酸', '青霉素', '青霉素G',
       '高水平庆大霉素', '高水平链霉素']

In [105]:
cohort_WITHMDR['ab_allna'] = pd.isna(cohort_WITHMDR[ab_list]).all(axis = 1).astype(int)

In [106]:
cohort_WITHMDR['org_sel_notest'] = cohort_WITHMDR[['org_sel','ab_allna']].apply(lambda x: 1 if x[0]== 1 and x[1]==1 else 0, axis =1)

In [107]:
cohort_WITHMDR.columns

Index(['PATIENT_ID', 'SPECIMEN', 'REQUESTED_DATE_TIME',
       'RESULTS_RPT_DATE_TIME', 'SUBJECT', 'ITEM_NO', 'REPORT_ITEM_NAME',
       'micro_name', 'merge_time', '万古霉素', '亚胺培南', '克林霉素', '利奈唑胺', '利福平',
       '厄它培南', '哌拉西林', '哌拉西林/他唑巴坦', '喹努普汀/达福普汀', '四环素', '复方新诺明', '头孢他啶',
       '头孢吡肟', '头孢呋辛', '头孢呋辛酯', '头孢唑啉', '头孢曲松', '头孢替坦', '头孢西丁', '妥布霉素',
       '左旋氧氟沙星', '庆大霉素', '替加环素', '替考拉宁', '氨曲南', '氨苄西林', '氨苄西林/舒巴坦', '环丙沙星',
       '磷霉素', '米诺环素', '红霉素', '美洛培南', '苯唑西林', '苯唑青霉素', '莫西沙星', '阿米卡星',
       '阿莫西林/棒酸', '青霉素', '青霉素G', '高水平庆大霉素', '高水平链霉素', 'time_hour',
       'resistant_ab_cat', 'MDR', 'org_category', 'org_pos', 'org_neg',
       'VISIT_ID', 'GENDER', 'AGE', 'ADMISSION_DATE_TIME',
       'DISCHARGE_DATE_TIME', 'ADMISSION_TYPE', 'ICU_TYPE',
       'TF_ADMISSION_DATE_TIME', 'TF_DISCHARGE_DATE_TIME', 'LOS_ICU',
       'firsthospmdrcharttime', 'firsthospmdrstoretime', 'firsthospmdrRAB',
       'firsthospmdrorgname', 'microtest_days', 'microresult_days',
       'microtest_inicu', 'microt

In [108]:
# cohort_WITHMDR[['PATIENT_ID', 'EXECUTE_DATE', 'SPECIMEN', 'SPCM_SAMPLE_DATE_TIME',
#        'REQUESTED_DATE_TIME', 'RESULTS_RPT_DATE_TIME', 'SUBJECT', 'ITEM_NO',
#        'REPORT_ITEM_NAME', 'micro_name', 'RESULT_DATE_TIME']].sort_values(by = ['PATIENT_ID', 'REQUESTED_DATE_TIME']).head(500).to_csv('sample.csv',encoding = 'ANSI', index = False)

In [109]:
clulture_neg_list = list(set(cohort_WITHMDR[(cohort_WITHMDR['micro_name'].str.contains('无|未',na=False))|\
              (cohort_WITHMDR['micro_name'].str.contains('培养阴性',na=False))|\
              (cohort_WITHMDR['micro_name'] == '阴性')].micro_name.to_list()))

In [110]:
#0代表阴性
cohort_WITHMDR['culture_negative'] = cohort_WITHMDR['micro_name'].apply(lambda x:0 if x in clulture_neg_list else 1)

In [111]:
cohort_WITHMDR.columns

Index(['PATIENT_ID', 'SPECIMEN', 'REQUESTED_DATE_TIME',
       'RESULTS_RPT_DATE_TIME', 'SUBJECT', 'ITEM_NO', 'REPORT_ITEM_NAME',
       'micro_name', 'merge_time', '万古霉素', '亚胺培南', '克林霉素', '利奈唑胺', '利福平',
       '厄它培南', '哌拉西林', '哌拉西林/他唑巴坦', '喹努普汀/达福普汀', '四环素', '复方新诺明', '头孢他啶',
       '头孢吡肟', '头孢呋辛', '头孢呋辛酯', '头孢唑啉', '头孢曲松', '头孢替坦', '头孢西丁', '妥布霉素',
       '左旋氧氟沙星', '庆大霉素', '替加环素', '替考拉宁', '氨曲南', '氨苄西林', '氨苄西林/舒巴坦', '环丙沙星',
       '磷霉素', '米诺环素', '红霉素', '美洛培南', '苯唑西林', '苯唑青霉素', '莫西沙星', '阿米卡星',
       '阿莫西林/棒酸', '青霉素', '青霉素G', '高水平庆大霉素', '高水平链霉素', 'time_hour',
       'resistant_ab_cat', 'MDR', 'org_category', 'org_pos', 'org_neg',
       'VISIT_ID', 'GENDER', 'AGE', 'ADMISSION_DATE_TIME',
       'DISCHARGE_DATE_TIME', 'ADMISSION_TYPE', 'ICU_TYPE',
       'TF_ADMISSION_DATE_TIME', 'TF_DISCHARGE_DATE_TIME', 'LOS_ICU',
       'firsthospmdrcharttime', 'firsthospmdrstoretime', 'firsthospmdrRAB',
       'firsthospmdrorgname', 'microtest_days', 'microresult_days',
       'microtest_inicu', 'microt

In [112]:
cohort_simp = cohort_WITHMDR[['PATIENT_ID',  'SPECIMEN', 'REQUESTED_DATE_TIME',  'culture_negative']].drop_duplicates()

In [113]:
cohort_simp_sum = cohort_simp.groupby(['PATIENT_ID',  'SPECIMEN', 'REQUESTED_DATE_TIME']).sum().reset_index()

In [114]:
cohort_simp_sum['culture_negative_tot'] = cohort_simp_sum['culture_negative'].apply(lambda x:0 if x==0 else 1)

In [115]:
cohort_simp_sum.drop(columns = ['culture_negative'] , inplace = True)

In [116]:
cohort_simp_sum[cohort_simp_sum['PATIENT_ID'] == 'S114268674']

,PATIENT_ID,SPECIMEN,REQUESTED_DATE_TIME,culture_negative_tot
99994,S114268674,引流液,2016-03-09 10:59:06,1
99995,S114268674,引流液,2016-03-09 10:59:18,0
99996,S114268674,引流液,2016-03-13 08:15:59,1
99997,S114268674,痰,2016-03-09 11:44:01,1
99998,S114268674,痰,2016-03-12 10:16:30,1
99999,S114268674,静脉血,2016-03-10 00:24:59,0
100000,S114268674,静脉血,2016-03-10 00:25:37,0
100001,S114268674,静脉血,2016-03-10 00:28:23,0
100002,S114268674,静脉血,2016-03-10 00:29:15,0
100003,S114268674,静脉血,2019-02-14 00:40:24,0


In [117]:
cohort_WITHMDR = pd.merge(cohort_WITHMDR,cohort_simp_sum, how = 'left', on = ['PATIENT_ID',  'SPECIMEN', 'REQUESTED_DATE_TIME'])

In [118]:
cohort_WITHMDR_part1 = cohort_WITHMDR[cohort_WITHMDR['culture_negative_tot'] == 0]

In [119]:
#cohort_WITHMDR[(cohort_WITHMDR['ab_allna'] == 1)&(cohort_WITHMDR['org_sel'] == 0)].micro_name.value_counts().to_csv('sel_noresult.csv',encoding = 'ANSI')

In [120]:
cohort_WITHMDR_part2 = cohort_WITHMDR[(cohort_WITHMDR['org_sel']==1) & (cohort_WITHMDR['ab_allna']!=1)]

In [121]:
cohort_WITHMDR_part1['PART'] = 'PART1'
cohort_WITHMDR_part2['PART'] = 'PART2'

In [122]:
cohort_WITHMDR_sel = pd.concat([cohort_WITHMDR_part1,cohort_WITHMDR_part2])

In [123]:
cohort_WITHMDR.to_csv('cohort_WITHMDR.csv', index = False)

In [124]:
cohort_WITHMDR_sel.to_csv('cohort_WITHMDR_sel.csv', index = False)

In [125]:
icustay_detail[icustay_detail['ICU_TYPE'] == 'ICU'][['PATIENT_ID','TF_ADMISSION_DATE_TIME']].drop_duplicates()

,PATIENT_ID,TF_ADMISSION_DATE_TIME
15,S108424989,2015-08-19 17:04:43
18,S108427755,2013-11-06 22:38:27
30,S108437842,2018-08-17 02:02:13
52,S108491846,2016-06-15 13:15:46
56,S108493128,2018-05-19 23:56:26
...,...,...
168384,S118047621,2019-02-01 18:31:43
168428,S118051322,2019-02-05 18:14:55
168448,S118052962,2019-02-12 20:48:05
168469,S118055349,2019-02-10 12:56:43


In [126]:
cohort_WITHMDR[['PATIENT_ID','TF_ADMISSION_DATE_TIME']].drop_duplicates()

,PATIENT_ID,TF_ADMISSION_DATE_TIME
0,S110236895,2009-01-10 15:40:00
1,S111326401,2009-07-02 17:00:00
2,S111326401,2009-09-02 17:20:00
3,S110676645,2008-11-17 18:14:08
5,S110183479,2008-12-05 11:46:58
...,...,...
648880,S113483336,NaT
648889,S118075204,2019-02-17 20:15:00
648892,S118072949,NaT
648904,S111214087,NaT


In [127]:
test = pd.merge(icustay_1stmdr, culture_mdr, how = 'left', on = ['PATIENT_ID'])

In [128]:
test[['PATIENT_ID','TF_ADMISSION_DATE_TIME']].drop_duplicates()

,PATIENT_ID,TF_ADMISSION_DATE_TIME
0,S108413368,2017-01-18 16:48:43
5,S108413368,2017-02-02 15:10:00
10,S108415620,2018-10-10 16:58:07
11,S108417789,2011-09-11 19:00:00
12,S108418984,2013-04-18 14:30:00
...,...,...
688702,S118088098,2019-02-20 13:50:00
688706,S118090924,2019-02-20 20:31:21
688707,S118091678,2019-02-20 13:54:48
688708,S118091792,2019-02-20 15:34:17
